In [259]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


#Keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten,Dropout
from keras.layers.convolutional import *
from keras.layers.recurrent import LSTM
from keras.utils.np_utils import to_categorical

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display
import pandas as pd

%matplotlib inline

In [260]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
#np.log(spec.T.astype(np.float32) + eps)

In [261]:
X_full = [None] * 641
labels = [None] * 641
i = 0 
for directory in os.listdir(os.getcwd() + "\\data\\"):
    for file in os.listdir(os.getcwd() + "\\data\\" + directory):
        samples, sample_rate  = librosa.load(os.getcwd() + "\\data\\" + directory + "\\" + file)
        freqs, times, spectrogram = log_specgram(samples, sample_rate)
        mean = np.mean(spectrogram, axis=0)
        std = np.std(spectrogram, axis=0)
        spectrogram = (spectrogram - mean) / std
        spectrogram = spectrogram.tolist()
        X_full[i] = spectrogram
        labels[i] = directory
        i = i + 1 

In [262]:
encoder = LabelEncoder()

labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

In [263]:
X_train, labels_train, X_valid, labels_valid = train_test_split(X_full, labels)

In [264]:
model = Sequential()
model.add(LSTM(units= 32, return_sequences=False, input_shape=(221, 98)))
model.add(Dense(units = 16, activation = "relu"))
model.add(Dense(units = 8, activation = "relu"))
model.add(Dense(units= 2, activation='sigmoid'))

In [265]:
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

In [266]:
model.fit_generator(X_train,
                    labels_train,
                    epochs = 10)

Epoch 1/10


TypeError: '<' not supported between instances of 'int' and 'list'

In [258]:
X_full

KeyboardInterrupt: 